In [10]:
import pandas as pd
from faker import Faker
import random

fake = Faker()

import os, requests, json
from dotenv import load_dotenv

load_dotenv()
GOOGLE_CALENDAR_API_KEY = os.getenv("GOOGLE_CALENDAR_API_KEY")

In [ ]:


url = "https://www.googleapis.com/calendar/v3/calendars/en.vietnamese%23holiday@group.v.calendar.google.com/events"
params = {
            "key": GOOGLE_CALENDAR_API_KEY,
            "timeMin" : "2022-01-01T00:00:00Z",
            "timeMax" : "2025-01-01T23:59:59Z",
            "singleEvents" : True,
            "orderBy" : "startTime",
            "maxResults" : 1000,
            "fields" : "items(start,end,summary)"
        }
response = requests.get(url, params=params)

with open("output.json", "w") as f:
    json.dump(response.json(), f, indent=4)

In [11]:
def convert_date_to_dateId(year, month, day):
    if month < 10:
        month = f'0{month}'
    if day < 10:
        day = f'0{day}'
    return f'{year}{month}{day}'

def convert_date(year, month, day):
    if month < 10:
        month = f'0{month}'
    if day < 10:
        day = f'0{day}'
    return f'{year}-{month}-{day}'

def get_holidays():
    url = "https://www.googleapis.com/calendar/v3/calendars/en.vietnamese%23holiday@group.v.calendar.google.com/events"
    params = {
                "key": GOOGLE_CALENDAR_API_KEY,
                "timeMin" : "2022-01-01T00:00:00Z",
                "timeMax" : "2025-01-01T23:59:59Z",
                "singleEvents" : True,
                "orderBy" : "startTime",
                "fields" : "items(start,end,summary)"
            }
    
    response = requests.get(url, params=params)
    return response.json()

def gen_date():
    start_date = pd.to_datetime('2022-01-01')
    end_date = pd.to_datetime('2024-12-31')
    date_range = pd.date_range(start=start_date, end=end_date)
    holidays = [(x["start"]["date"], x["summary"]) for x in get_holidays()["items"]]
    
    dateID = []
    day = []
    week = []
    month = []
    quarter = []
    year = []
    isHoliday = []
    isWeekend = []
    holidayName = []

    for date in date_range:
        dateID.append(convert_date_to_dateId(date.year, date.month, date.day))
        day.append(date.day)
        week.append(date.isocalendar()[1])
        month.append(date.month)
        quarter.append((date.month - 1) // 3 + 1)
        year.append(date.year)
        if convert_date(date.year, date.month, date.day) in [x[0] for x in holidays]:
            isHoliday.append(1)
            holidayName.append([x[1] for x in holidays if x[0] == convert_date(date.year, date.month, date.day)][0])
        else:
            isHoliday.append(0)
            holidayName.append("")
        isWeekend.append(1 if date.weekday() >= 5 else 0)

    df = pd.DataFrame({
        "dateID": dateID,
        "day": day,
        "week": week,
        "month": month,
        "quarter": quarter,
        "year": year,
        "isHoliday": isHoliday,
        "isWeekend": isWeekend,
        "holidayName": holidayName
    })    

    return df

gen_date().to_csv("test_output.csv", index=False)

In [16]:
def gen_review(num_reviews, num_customers, product_df, date_df):
    positive_comments = [
        # Positive Comments
        "It was absolutely delicious and full of flavor!",
        "The presentation of the dish was stunning.",
        "It were refreshing and perfectly balanced.",
        "The service was excellent, and the staff was very friendly.",
        "It was heavenly and melted in my mouth.",
        "The portion sizes were generous and satisfying.",
        "The ambiance of the restaurant was cozy and inviting.",
        "The ingredients tasted fresh and high-quality.",
        "It was rich and aromatic, just perfect.",
        "The menu had a great variety of options to choose from.",
        "It was cooked to perfection, al dente!",
        "It was tender and cooked exactly as requested.",
        "It were creative and expertly crafted.",
        "It was warm and comforting, just what I needed.",
        "It was fresh and beautifully presented.",
        "The flavors were well-balanced and complemented each other.",
        "The staff went above and beyond to make our experience special.",
        "It was served hot and on time.",
        "The restaurant had a great vibe and atmosphere.",
        "It was juicy and packed with flavor.",
        "The service was quick and efficient.",
        "It was creamy and had a rich flavor.",
        "It was worth every penny, great value for money.",
        "The restaurant was clean and well-maintained.",
        "The staff was knowledgeable about the menu and made great recommendations.",
        "The flavors were authentic and reminded me of home.",
        "It were served at the perfect temperature.",
        "The restaurant had a unique and creative menu.",
        "The service was attentive without being intrusive.",
        "It was flavorful and left me wanting more.",
        "The ambiance was perfect for a romantic dinner.",
        "It was the best thing I’ve had in a long time.",
        "The staff was polite and made us feel welcome.",
        "The restaurant exceeded my expectations in every way."
    ]
    neutral_comments = [
        # Neutral Comments
        "It was decent, but nothing extraordinary.",
        "The service was okay, but it could have been faster.",
        "It were fine, but a bit too sweet for my taste.",
        "The ambiance was nice, but the music was a bit loud.",
        "It was fine, but I’ve had better elsewhere.",
        "The menu had a decent variety, but nothing stood out.",
        "The coffee was okay, but not as strong as I like it.",
        "It was served warm, but not hot.",
        "It was good, but slightly overcooked.",
        "It was fine, but it could have been more tender.",
        "It were okay, but not very creative.",
        "It was fine, but it wasn’t very fresh.",
        "The flavors were okay, but they didn’t wow me.",
        "The staff was polite, but not very attentive.",
        "It was served on time, but the presentation was lacking.",
        "The restaurant was nice, but it felt a bit crowded.",
        "It was fine, but the dressing was too heavy.",
        "The service was fine, but it could have been more friendly.",
        "It was good, but it felt a bit overpriced.",
        "The chef did a decent job, but the dishes lacked creativity.",
        "The appetizers were fine, but they were a bit bland.",
        "The restaurant was clean, but the decor was outdated.",
        "The staff was okay, but they seemed a bit rushed.",
        "The flavors were fine, but they didn’t feel authentic.",
        "It were fine, but they took too long to arrive.",
        "It was cooked well, but it lacked seasoning.",
        "The restaurant had a decent menu, but it wasn’t very unique.",
        "The service was fine, but it wasn’t very memorable.",
        "It was okay, but it didn’t leave a lasting impression.",
        "The ambiance was fine, but it felt a bit generic."
    ]    
    negative_comments = [
        # Negative Comments
        "It was bland and lacked flavor.",
        "The service was slow and unprofessional.",
        "It were watered down and disappointing.",
        "The ambiance was ruined by loud noise and poor lighting.",
        "The portion sizes were too small for the price.",
        "It was cold and unappetizing.",
        "It was overcooked and mushy.",
        "It was tough and hard to chew.",
        "It were poorly made and tasted bad.",
        "The staff was rude and inattentive.",
        "It took too long to arrive and wasn’t worth the wait.",
        "The restaurant was dirty and poorly maintained.",
        "It was wilted and unappealing.",
        "It tasted off and wasn’t fresh.",
        "It was overly sweet and artificial-tasting.",
        "The service was chaotic and disorganized.",
        "It was overpriced and not worth the money.",
        "The chef didn’t seem to put much effort into the dishes.",
        "The restaurant was cramped and uncomfortable.",
        "The staff was unhelpful and seemed annoyed.",
        "The flavors were unbalanced and unpleasant.",
        "It was too greasy and heavy.",
        "It were served at the wrong temperature.",
        "Its platter was disappointing and lacked creativity.",
        "It was undercooked and unsafe to have.",
        "The restaurant had a limited and uninspired menu.",
        "The service was inattentive and frustrating.",
        "It was forgettable and not worth recommending.",
        "The ambiance was dull and uninviting.",
        "It was poorly plated and unappetizing.",
        "The staff was unprofessional and unwelcoming.",
        "The restaurant failed to meet even basic expectations.",
        "It was a complete letdown and not enjoyable."
    ]
    
    product_list = []
    customer_list = []
    rating_list = []
    comment_list = []
    date_list = []

    for i in range(num_reviews):
        productId = random.randint(1, len(product_df))
        customerId = random.randint(1, num_customers)
        rating = random.randint(1, 10)
        if rating <= 4:
            comment = random.choice(negative_comments)
        elif rating <= 7:
            comment = random.choice(neutral_comments)
        else:
            comment = random.choice(positive_comments)
        date = random.choice(date_df['dateID'].tolist())
        product_list.append(productId)
        customer_list.append(customerId)
        rating_list.append(rating)
        comment_list.append(comment)
        date_list.append(date)

    df = pd.DataFrame({
        'productId': product_list,
        'customerId': customer_list,
        'rating': rating_list,
        'comment': comment_list,
        'date': date_list
    })

    rating_dict = {}

    for i in range(len(df)):
        productId = df.iloc[i]['productId']
        rating = df.iloc[i]['rating']
        if productId in rating_dict:
            rating_dict[productId].append(rating)
        else:
            rating_dict[productId] = [rating]

    for x,y in rating_dict.items():
        product_df.loc[x-1, 'rating'] = round(sum(y) / len(y), 2)

    return df

product = pd.read_csv("product.csv")
gen_review(1000, 100, product, pd.read_csv("test_output.csv")).to_csv("reviews.csv", index=False)

product

,product_id,product_category,product_type,product_name,product_description,rating,unit_price,image,discount,state
0,1,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,5.08,14.40,NaN,0,available
1,2,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,4.60,14.40,NaN,0,available
2,3,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,5.67,11.80,NaN,0,available
3,4,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,6.08,16.36,NaN,0,available
4,5,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,5.20,12.00,NaN,0,available
...,...,...,...,...,...,...,...,...,...,...
79,80,Flavours,Regular syrup,Chocolate syrup,Bursting with chocolate flavour,6.40,0.04,NaN,0,available
80,81,Coffee,Specialty coffee,Rio Nights,2 shots of Ouro Brasilerio and pure cane sugar...,5.09,1.20,NaN,0,available
81,82,Coffee,Barista Espresso,Ouro Brasileiro shot,From Rio,6.15,0.60,NaN,0,available
82,83,Coffee,Barista Espresso,Ouro Brasileiro shot promo,Ouro promo,6.14,0.42,NaN,0,available
